In [21]:
import copy
import math 
import random
import sys

In [22]:
fp=open('input.txt','r')
nq=int(fp.readline().rstrip())
q=[]
for _ in range(0,(nq)):
    q.append(str(fp.readline().rstrip()))

ns=int(fp.readline().rstrip())
s=[]
for _ in range(0,(ns)):
    s.append(str(fp.readline().rstrip()))

In [23]:
class phar:
    def __init__(self,sign,name,para):
        self.sign=sign
        self.name=name
        self.para=para

pos={}
neg={}

def preprocess(q,nq,s,ns):
    sentence=[]
    cnter=0
    for sent in s:
        cnter+=1
        sent=sent.replace(" ","")
        sent=sent.split('=>')
        if(len(sent)>1):
            andsent=sent[0].split('&')
            for i in range(0,len(andsent)):            
                if(andsent[i][0]=='~'):
                    andsent[i]=andsent[i][1:]
                else:
                    andsent[i]="~"+andsent[i]         
            del(sent[0])
            sent+=(andsent)
        mainsent=[]
        for ss in sent:        
            sign=1
            name=''
            para=[]
            temp=ss.split('(')       
            if(temp[0][0]=='~'):
                sign=0
                name=temp[0][1:]
            else:
                name=temp[0]
            para=temp[1].split(',')
            para[-1]=para[-1][0:len(para[-1])-1]
            for i in range(0,len(para)):
                
                if para[i][0].islower():
                    para[i]+=str(cnter)
            a=phar(sign,name,para)
            mainsent.append(a)
            
            if(sign==1):
                if(name in pos):
                    pos[name].append({cnter-1:para})
                else:
                    pos[name]=[{cnter-1:para}]
            else:
                if(name in neg):
                    neg[name].append({cnter-1:para})
                else:
                    neg[name]=[{cnter-1:para}]
        sentence.append(mainsent)
        
    return sentence

def unify(x,y,subs):
    if subs is None:
        return {}
    elif y==x:
        return subs
    elif type(x)==str and x[0].islower():
        return unify_var(x,y,subs)
    elif type(y)==str and y[0].islower():
        return unify_var(y,x,subs)
    elif isinstance(x,phar) and isinstance(y,phar):
        if x.name!=y.name or x.sign==y.sign or len(x.para)!=len(y.para):
            return {}
        else:
            for i in range(0,len(x.para)):
                subs=unify(x.para[i],y.para[i],subs)
            return subs
    else:
        return {}
    
def unify_var(x,y,subs):
    if x in subs:
        return unify(subs[x],y,subs)
    elif isinstance(y,phar) and y in subs:
        return unify(x,subs[y],subs)
    else:
        subs[x]=y
        #print(x," replaced with ",y)
        return subs

    
def res(sentence,curq,pos1,neg1):
    
    print(pos1)
    print(neg1)
    sign=0
    name=''
    para=[]
    temp=curq.split('(')       
    if(temp[0][0]=='~'):
        sign=1
        name=temp[0][1:]
    else:
        name=temp[0]
    para=temp[1].split(',')
    para[-1]=para[-1][0:len(para[-1])-1]
    
    for i in range(0,len(para)):
        if para[i][0].islower():
            para[i]+=str(0)
    
    if(sign==1):
        if(name in pos):
            pos1[name].append({len(sentence):para})
        else:
            pos1[name]=[{len(sentence):para}]
    else:
        if(name in neg):
            neg1[name].append({len(sentence):para})
        else:
            neg1[name]=[{len(sentence):para}]
                    
    #print(pos1)
    #print(neg1)
    cur_q=phar(sign,name,para)
        
    sentence_new=sentence+[[cur_q]]
    resolved=[[0 for _ in range(0,len(sentence_new))] for __ in range(0,len(sentence_new))]
    counter=len(sentence_new)
    while(1):
        new=[]
        
        #print(resolved)
        #print("----------------------------------------------------------------------")
        #print("length of KB is ",len(sentence_new))
        for i in range(0,len(sentence_new)):
            for j in range(i+1,len(sentence_new)):
                ##print("resolved value:",resolved[i][j])
                #print("i",i,"j",j)
                #print("res",resolved[i][j])
                if(resolved[i][j]==1):
                    #print("Continuinf")
                    continue
                flag=0
                for s1 in sentence_new[i]:
                    for s2 in sentence_new[j]:
                        #print("i",i,"-",s1.sign,s1.name,s1.para)
                        #print("j",j,"-",s2.sign,s2.name,s2.para)
                        if s1.name==s2.name and s1.sign!=s2.sign:
                            #tempo=unify(sentence_new[i],s1,sentence_new[j],s2)
                            cnt=1
                            for pp in range(0,len(s1.para)):
                                if(s1.para[pp]!=s2.para[pp]):
                                    cnt=0
                            if(cnt):
                                #print("**************")
                                #print("UNIFY CONSTANTS")
                                
                                #print("sent 1")
                                #for o in sentence_new[i]:
                                    #print(o.sign,o.name,o.para)
                                #print("unif pred")
                                #print(s1.sign,s1.name,s1.para)
                            
                                #print("sent 2")
                                #for o in sentence_new[j]:
                                    #print(o.sign,o.name,o.para)
                                #print("unif pred")
                                #print(s2.sign,s2.name,s2.para)
                                #print("LENGTHS BEFORE")
                                #print(len(sentence_new[i]),len(sentence_new[j]))
                            
                            
                                unif_sent1=copy.deepcopy(sentence_new[i])
                                idx=-1
                                for k in range(0,len(unif_sent1)):
                                    if(unif_sent1[k].sign==s1.sign and unif_sent1[k].name==s1.name and unif_sent1[k].para==s1.para):
                                        idx=k
                                del unif_sent1[idx]
                                """
                                for o in unif_sent1:
                                    if(o.sign==s1.sign and o.name==s1.name and o.para==s1.para):
                                        unif_sent1.remove(o)
                                """
                                unif_sent2=copy.deepcopy(sentence_new[j])
                                
                                idx=-1
                                for kk in range(0,len(unif_sent2)):
                                    if(unif_sent2[kk].sign==s2.sign and unif_sent2[kk].name==s2.name and unif_sent2[kk].para==s2.para):
                                        idx=kk
                                del unif_sent2[idx]
                                
                                """
                                for o in unif_sent2:
                                    if(o.sign==s2.sign and o.name==s2.name and o.para==s2.para):
                                        unif_sent2.remove(o)
                                """
                                unif_sent=[]
                                unif_sent+=unif_sent1
                                unif_sent+=unif_sent2
                                
                                f=0
                                for ob in unif_sent:
                                    for va in range(0,len(ob.para)):
                                        if(ob.para[va][0].islower()):
                                            ob.para[va]=ob.para[va]+str("_")+str(counter)
                                            f=1
                                if(f):
                                    counter+=1
                                
                                dupli=set()
                                for i1 in range(0,len(unif_sent)):
                                    for j1 in range(i1+1,len(unif_sent)):
                                        
                                        if(unif_sent[i1].sign==unif_sent[j1].sign and unif_sent[i1].name==unif_sent[j1].name):
                                            tt=1
                                            for t in range(0,len(unif_sent[i1].para)):
                                                if((unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].islower()) or
                                                (unif_sent[i1].para[t].islower() and unif_sent[j1].para[t].isupper()) or
                                                (unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].isupper() and 
                                                unif_sent[i1].para[t]!=unif_sent[j1].para[t])):
                                                    tt=0
                                            if(tt):
                                                dupli.add(j1)
                                
                                del_cnt1=0
                                for idp in dupli:
                                    del unif_sent[idp-del_cnt1]
                                    del_cnt1+=1
                                    
                                #print("dupli")
                                #if(del_cnt1!=0):
                                    #print(dupli)
                                    
                                cance=1
                                while(cance):
                                    cance=0
                                    for i1 in range(0,len(unif_sent)):
                                        while_break=0
                                        for j1 in range(i1+1,len(unif_sent)):
                                        
                                            if(unif_sent[i1].sign!=unif_sent[j1].sign and unif_sent[i1].name==unif_sent[j1].name):
                                                tt=1
                                                for t in range(0,len(unif_sent[i1].para)):
                                                    if((unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].islower()) or
                                                    (unif_sent[i1].para[t].islower() and unif_sent[j1].para[t].isupper()) or
                                                    (unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].isupper() and 
                                                    unif_sent[i1].para[t]!=unif_sent[j1].para[t])):
                                                        tt=0
                                                if(tt):
                                                    del (unif_sent[i1])
                                                    del (unif_sent[1-1])
                                                    cance=1
                                                    while_break=0
                                                    break
                                        if(while_break):
                                            break
                                
                                
                                #print("UNIFY CONSTANTS")
                                
                                #print("\nsent 1")
                                #for o in sentence_new[i]:
                                    #print(o.sign,o.name,o.para)
                                #print("unif pred")
                                #print(s1.sign,s1.name,s1.para)
                            
                                #print("\nsent 2")
                                #for o in sentence_new[j]:
                                    #print(o.sign,o.name,o.para)
                                #print("\nunif pred")
                                #print(s2.sign,s2.name,s2.para)
                                #print("LENGTHS BEFORE")
                                #print(len(sentence_new[i]),len(sentence_new[j]))
                                
                                
                                
                                
                                
                                if(len(unif_sent)==0):
                                    """
                                    print("----------------------------------------------------")
                                    for ss in sentnece_new[i]:
                                        print(ss.sign,ss.name,ss.para)
                                    for ss in sentnece_new[j]:
                                        print(ss.sign,ss.name,ss.para)
                                    print("----------------------------------------------------")
                                    
                                    for _ in range(0,len(sentence_new)):
                                        print("Sentence:",_)
                                        for __ in sentence_new[_]:
                                            print(__.sign,__.name,__.para)
                                    """        
                                    return("TRUE")
                                
                                
                                #print("\nUnif sent")
                                #inter=[_ for _ in range(0,len(sentence_new))]
                                inter=[]
                                if(unif_sent[0].sign==1):
                                    inter=[list(_.keys())[0] for _ in pos1[unif_sent[0].name]]
                                else:
                                    inter=[list(_.keys())[0] for _ in neg1[unif_sent[0].name]]
                                inter_flag=1
                                for o in unif_sent:
                                    #print(o.sign,o.name,o.para)
                                    if(o.sign==1):
                                        if(o.name in pos1):
                                            inter=list(set(inter)&set(list(_.keys())[0] for _ in pos1[o.name]))
                                        else:
                                            inter_flag=0
                                    else:
                                        if(o.name in neg1):
                                            inter=list(set(inter)&set(list(_.keys())[0] for _ in neg1[o.name]))
                                        else:
                                            inter_flag=0
                                            
                                poda=[-1 for _ in range(0,len(inter))]
                                poda_cnt=0
                                #print("inter",inter)
                                if(inter_flag and len(inter)!=0):
                                    for loc in inter:
                                        if(loc>=len(sentence_new)):
                                            
                                            if(len(new[loc-len(sentence_new)])==len(unif_sent)):
                                                
                                                poda[poda_cnt]=1
                                                for loc_sent in new[loc-len(sentence_new)]:
                                                    for un_sent in unif_sent:
                                                        if(loc_sent.sign==un_sent.sign and loc_sent.name==un_sent.name):
                                                            #WARNING
                                                            #same sentence can have 2  predicate of same name and diff args too.
                                                            for ch in range(0,len(loc_sent.para)):
                                                                if (((loc_sent.para[ch][0]).islower() and (un_sent.para[ch][0]).isupper()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).islower()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).isupper() and loc_sent.para[ch]!=un_sent.para[ch])):
                                                                    poda[poda_cnt]=0
                                            else:
                                                poda[poda_cnt]=0
                                            
                                            
                                        else:    
                                            if(len(sentence_new[loc])==len(unif_sent)):
                                                poda[poda_cnt]=1
                                                for loc_sent in sentence_new[loc]:
                                                    for un_sent in unif_sent:
                                                        if(loc_sent.sign==un_sent.sign and loc_sent.name==un_sent.name):
                                                            #same sentence can have 2  predicate of same name
                                                            for ch in range(0,len(loc_sent.para)):
                                                                if (((loc_sent.para[ch][0]).islower() and (un_sent.para[ch][0]).isupper()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).islower()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).isupper() and loc_sent.para[ch]!=un_sent.para[ch])):
                                                                    poda[poda_cnt]=0
                                            else:
                                                poda[poda_cnt]=0
                                        poda_cnt+=1
                                #print("poda",poda)        
                                if((1 in poda)):
                                    #print("Sentence already in KB. Skipping duplicate")
                                    continue        
                                            
                                
                                #print("*********************************")
                                
                                for o in unif_sent:
                                    #print(o.sign,o.name,o.para)
                                    if(o.sign==1):
                                        if(o.name in pos1):
                                            pos1[o.name].append({len(sentence_new)+len(new):o.para})
                                        else:
                                            pos1[o.name]=[{len(sentence_new)+len(new):o.para}]
                                    else:
                                        if(o.name in neg1):
                                            neg1[o.name].append({len(sentence_new)+len(new):o.para})
                                        else:
                                            neg1[o.name]=[{len(sentence_new)+len(new):o.para}]
                                new.append(unif_sent)
                                   
                                #print("NEW LENGTHS AFTER")
                                #print(len(sentence_new[i]),len(sentence_new[j]),len(unif_sent))
                                """
                                if (tempo==False):
                                    continue
                                elif(tempo==[]):
                                    return ("TRUE")
                                new.append(tempo)
                                """
                            
                                    #print(new)
                                flag=1
                            
                            else:
                                sub={}
                                sub=unify(s1,s2,{})
                                if(len(sub)==0):
                                    continue
                                #print("**************")
                                #print(sub)
                                #print("sent 1")
                                #for o in sentence_new[i]:
                                    #print(o.sign,o.name,o.para)
                                #print("unif pred")
                                #print(s1.sign,s1.name,s1.para)
                            
                                #print("sent 2")
                                #for o in sentence_new[j]:
                                    #print(o.sign,o.name,o.para)
                                #print("unif pred")
                                #print(s2.sign,s2.name,s2.para)
                                #print("LENGTHS BEFORE")
                                #print(len(sentence_new[i]),len(sentence_new[j]))
                            
                            
                                unif_sent1=copy.deepcopy(sentence_new[i])
                                idx=-1
                                for k in range(0,len(unif_sent1)):
                                    if(unif_sent1[k].sign==s1.sign and unif_sent1[k].name==s1.name and unif_sent1[k].para==s1.para):
                                        idx=k
                                del unif_sent1[idx]
                                """
                                for o in unif_sent1:
                                    if(o.sign==s1.sign and o.name==s1.name and o.para==s1.para):
                                        unif_sent1.remove(o)
                                """
                                unif_sent2=copy.deepcopy(sentence_new[j])
                                
                                idx=-1
                                for kk in range(0,len(unif_sent2)):
                                    if(unif_sent2[kk].sign==s2.sign and unif_sent2[kk].name==s2.name and unif_sent2[kk].para==s2.para):
                                        idx=kk
                                del unif_sent2[idx]
                                
                                """
                                for o in unif_sent2:
                                if(o.sign==s2.sign and o.name==s2.name and o.para==s2.para):
                                    unif_sent2.remove(o)
                                """
                                unif_sent=[]
                                unif_sent+=unif_sent1
                                unif_sent+=unif_sent2
                                for r in unif_sent:
                                    for kkk in range(0,len(r.para)):
                                        if (r.para[kkk] in sub):
                                            r.para[kkk]=sub[r.para[kkk]]
                                            
                                f=0
                                for ob in unif_sent:
                                    for va in range(0,len(ob.para)):
                                        if(ob.para[va][0].islower()):
                                            ob.para[va]=ob.para[va]+str("_")+str(counter)
                                            f=1
                                if(f):
                                    counter+=1
                                
                                
                                dupli=set()
                                for i1 in range(0,len(unif_sent)):
                                    for j1 in range(i1+1,len(unif_sent)):
                                        
                                        if(unif_sent[i1].sign==unif_sent[j1].sign and unif_sent[i1].name==unif_sent[j1].name):
                                            tt=1
                                            for t in range(0,len(unif_sent[i1].para)):
                                                if((unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].islower()) or
                                                (unif_sent[i1].para[t].islower() and unif_sent[j1].para[t].isupper()) or
                                                (unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].isupper() and 
                                                unif_sent[i1].para[t]!=unif_sent[j1].para[t])):
                                                    tt=0
                                            if(tt):
                                                dupli.add(j1)
                                
                                del_cnt1=0
                                for idp in dupli:
                                    del unif_sent[idp-del_cnt1]
                                    del_cnt1+=1
                                
                                #print("dupli")
                                #if(del_cnt1!=0):
                                    #print(dupli)
                                    
                                    
                                cance=1
                                while(cance):
                                    cance=0
                                    for i1 in range(0,len(unif_sent)):
                                        while_break=0
                                        for j1 in range(i1+1,len(unif_sent)):
                                        
                                            if(unif_sent[i1].sign!=unif_sent[j1].sign and unif_sent[i1].name==unif_sent[j1].name):
                                                tt=1
                                                for t in range(0,len(unif_sent[i1].para)):
                                                    if((unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].islower()) or
                                                    (unif_sent[i1].para[t].islower() and unif_sent[j1].para[t].isupper()) or
                                                    (unif_sent[i1].para[t].isupper() and unif_sent[j1].para[t].isupper() and 
                                                    unif_sent[i1].para[t]!=unif_sent[j1].para[t])):
                                                        tt=0
                                                if(tt):
                                                    del (unif_sent[i1])
                                                    del (unif_sent[1-1])
                                                    cance=1
                                                    while_break=0
                                                    break
                                        if(while_break):
                                            break
                                
                                
                                
                                
                                """
                                print("UNIFYING VARIABLE")
                                print(sub)
                                print("\nsent 1")
                                for o in sentence_new[i]:
                                    print(o.sign,o.name,o.para)
                                #print("unif pred")
                                #print(s1.sign,s1.name,s1.para)
                            
                                print("\nsent 2")
                                for o in sentence_new[j]:
                                    print(o.sign,o.name,o.para)
                                #print("unif pred")
                                #print(s2.sign,s2.name,s2.para)
                                #print("LENGTHS BEFORE")
                                #print(len(sentence_new[i]),len(sentence_new[j]))
                                """
                                if(len(unif_sent)==0):
                                    """
                                    print("----------------------------------------------------")
                                    for ss in sentence_new[i]:
                                        print(ss.sign,ss.name,ss.para)
                                    for ss in sentence_new[j]:
                                        print(ss.sign,ss.name,ss.para)
                                    print("----------------------------------------------------")  
                                    
                                    for _ in range(0,len(sentence_new)):
                                        print("Sentence:",_)
                                        for __ in sentence_new[_]:
                                            print(__.sign,__.name,__.para)
                                    """
                                    return("TRUE")
                                
                                
                                #print("\nUnif sent")
                                #inter=[_ for _ in range(0,len(sentence_new))]
                                inter=[]
                                if(unif_sent[0].sign==1):
                                    inter=[list(_.keys())[0] for _ in pos1[unif_sent[0].name]]
                                else:
                                    inter=[list(_.keys())[0] for _ in neg1[unif_sent[0].name]]
                                inter_flag=1
                                for o in unif_sent:
                                    #print(o.sign,o.name,o.para)
                                    if(o.sign==1):
                                        if(o.name in pos1):
                                            inter=list(set(inter)&set(list(_.keys())[0] for _ in pos1[o.name]))
                                        else:
                                            inter_flag=0
                                    else:
                                        if(o.name in neg1):
                                            inter=list(set(inter)&set(list(_.keys())[0] for _ in neg1[o.name]))
                                        else:
                                            inter_flag=0
                                
                                
                                
                                
                                poda=[-1 for _ in range(0,len(inter))]
                                poda_cnt=0
                                #print("inter",inter)
                                if(inter_flag and len(inter)!=0):
                                    for loc in inter:
                                        if(loc>=len(sentence_new)):
                                            
                                            if(len(new[loc-len(sentence_new)])==len(unif_sent)):
                                                
                                                poda[poda_cnt]=1
                                                for loc_sent in new[loc-len(sentence_new)]:
                                                    for un_sent in unif_sent:
                                                        if(loc_sent.sign==un_sent.sign and loc_sent.name==un_sent.name):
                                                            #same sentence can have 2  predicate of same name
                                                            for ch in range(0,len(loc_sent.para)):
                                                                if (((loc_sent.para[ch][0]).islower() and (un_sent.para[ch][0]).isupper()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).islower()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).isupper() and loc_sent.para[ch]!=un_sent.para[ch])):
                                                                    poda[poda_cnt]=0
                                            else:
                                                poda[poda_cnt]=0
                                            
                                            
                                        else:    
                                            if(len(sentence_new[loc])==len(unif_sent)):
                                                poda[poda_cnt]=1
                                                for loc_sent in sentence_new[loc]:
                                                    for un_sent in unif_sent:
                                                        if(loc_sent.sign==un_sent.sign and loc_sent.name==un_sent.name):
                                                            #same sentence can have 2  predicate of same name
                                                            for ch in range(0,len(loc_sent.para)):
                                                                if (((loc_sent.para[ch][0]).islower() and (un_sent.para[ch][0]).isupper()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).islower()) or ((loc_sent.para[ch][0]).isupper() and (un_sent.para[ch][0]).isupper() and loc_sent.para[ch]!=un_sent.para[ch])):
                                                                    poda[poda_cnt]=0
                                            else:
                                                poda[poda_cnt]=0
                                        poda_cnt+=1
                                #print("poda",poda)        
                                if((1 in poda)):
                                    #print("Sentence already in KB. Skipping duplicate")
                                    continue 
                                
                                
                                #print("******************************")
                                
                                
                                for o in unif_sent:
                                    #print(o.sign,o.name,o.para)
                                    if(o.sign==1):
                                        if(o.name in pos1):
                                            pos1[o.name].append({len(sentence_new)+len(new):o.para})
                                        else:
                                            pos1[o.name]=[{len(sentence_new)+len(new):o.para}]
                                    else:
                                        if(o.name in neg1):
                                            neg1[o.name].append({len(sentence_new)+len(new):o.para})
                                        else:
                                            neg1[o.name]=[{len(sentence_new)+len(new):o.para}]
                                            
                                new.append(unif_sent)
                                

                                #print("NEW LENGTHS AFTER")
                                #print(len(sentence_new[i]),len(sentence_new[j]),len(unif_sent))
                                """
                                if (tempo==False):
                                    continue
                                elif(tempo==[]):
                                    return ("TRUE")
                                new.append(tempo)
                                """
                                
                                #print(new)
                                flag=1
                if(flag):
                    resolved[i][j]=1
                #print(i,j,"flag",flag,"resolved",resolved[i][j])
                #print(resolved)
    
        """
        for pod in new:
            print("sentnece")
            for poda in pod:
                print(poda.sign,poda.name,poda.para)
        """
    
        #print("pos1",pos1)
        #print("neg1",neg1)
        if(len(new)==0):
            #for _ in range(0,len(sentence_new)):
                #print("Sentence:",_)
                #for __ in sentence_new[_]:
                    #print(__.sign,__.name,__.para)
            return("FALSE")
        sentence_new+=new
        L=len(resolved)
        
        for i in range(0,L):
            resolved[i]+=[0 for _ in range(0,len(new))]
        resolved+=[[0 for _ in range(0,len(new)+L)] for __ in range(0,len(new))]

In [24]:
        
sentence=preprocess(q,nq,s,ns)
ret=[]
for curq in q:
    pos1=copy.deepcopy(pos)
    neg1=copy.deepcopy(neg)
    ret.append(res(sentence,curq,pos1,neg1))

with open("output.txt","w") as f:
    for ret1 in ret:
        print(ret1,file=f)

{'Sick': [{0: ['Hehe']}, {7: ['Kiran']}], 'Eat': [{1: ['Med', 'Xixi']}, {4: ['Med', 'x5']}], 'Cure': [{2: ['x3']}], 'Shop': [{3: ['Med']}], 'Dumb': [{5: ['Kiran']}], 'Crazy': [{6: ['Kiran']}]}
{'Sick': [{2: ['x3']}, {3: ['x4']}, {4: ['x5']}], 'Eat': [{2: ['Med', 'x3']}], 'Shop': [{4: ['Med']}]}
{'Sick': [{0: ['Hehe']}, {7: ['Kiran']}], 'Eat': [{1: ['Med', 'Xixi']}, {4: ['Med', 'x5']}], 'Cure': [{2: ['x3']}], 'Shop': [{3: ['Med']}], 'Dumb': [{5: ['Kiran']}], 'Crazy': [{6: ['Kiran']}]}
{'Sick': [{2: ['x3']}, {3: ['x4']}, {4: ['x5']}], 'Eat': [{2: ['Med', 'x3']}], 'Shop': [{4: ['Med']}]}
